In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/faiss-data/algerian_legal(joconstitutionpenalecivilcommercefamille) embedder_ dangvantuan-sentence-camembert-large.faiss
/kaggle/input/faiss-data/algerian_legal(joconstitutionpenalecivilcommercefamille) embedder_ dangvantuan-sentence-camembert-large_meta.json
/kaggle/input/faiss-data/algerian_legal(joconstitutionpenalecivilcommercefamille) embedder_ dangvantuan-sentence-camembert-large_docs.json


In [2]:
!pip install -q transformers torch sentence-transformers faiss-cpu accelerate bitsandbytes sacremoses sentencepiece langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 85.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 42.5 MB/s eta 0:00:00


In [3]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from typing import List, Dict, Tuple, Optional
from pathlib import Path
import warnings
from langdetect import detect, LangDetectException
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")

2025-12-25 17:05:32.586744: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766682332.806458      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766682332.867842      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766682333.365913      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766682333.365954      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766682333.365957      55 computation_placer.cc:177] computation placer alr

✅ All libraries imported successfully!
✅ PyTorch version: 2.8.0+cu126
✅ CUDA available: True


In [4]:
# Paths (Update these to match your uploaded files)
INDEX_PATH = "/kaggle/input/faiss-data/algerian_legal(joconstitutionpenalecivilcommercefamille) embedder_ dangvantuan-sentence-camembert-large.faiss"
DOCS_PATH = INDEX_PATH.replace('.faiss', '_docs.json')
META_PATH = INDEX_PATH.replace('.faiss', '_meta.json')

# Models
EMBEDDING_MODEL = "dangvantuan/sentence-camembert-large"
LLM_MODEL = "bofenghuang/vigogne-2-7b-chat"

# Translation Model - Best for Arabic-French legal translation
# NLLB-200 is Meta's state-of-the-art multilingual model, excellent for Arabic-French
TRANSLATION_MODEL = "facebook/nllb-200-distilled-600M"  # Good balance of quality and speed
# Alternative: "facebook/nllb-200-1.3B" for better quality but slower
# Alternative: "Helsinki-NLP/opus-mt-ar-fr" and "Helsinki-NLP/opus-mt-fr-ar" (lighter but less accurate)

# Settings
USE_4BIT_QUANTIZATION = True
TOP_K_RETRIEVAL = 3

print("✅ Configuration set!")
print(f"  Embedding Model: {EMBEDDING_MODEL}")
print(f"  LLM Model: {LLM_MODEL}")
print(f"  Translation Model: {TRANSLATION_MODEL}")

✅ Configuration set!
  Embedding Model: dangvantuan/sentence-camembert-large
  LLM Model: bofenghuang/vigogne-2-7b-chat
  Translation Model: facebook/nllb-200-distilled-600M


In [5]:
print("Loading translation model...")

translation_tokenizer = AutoTokenizer.from_pretrained(
    TRANSLATION_MODEL,
    src_lang="fra_Latn",
    tgt_lang="ara_Arab"
)
translation_model = AutoModelForSeq2SeqLM.from_pretrained(
    TRANSLATION_MODEL,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

# Get language token IDs for NLLB
def get_lang_token_id(tokenizer, lang_code):
    """Get the token ID for a language code"""
    # For NLLB models, language codes are special tokens
    try:
        return tokenizer.convert_tokens_to_ids(lang_code)
    except:
        # Alternative method
        return tokenizer.lang_code_to_id[lang_code] if hasattr(tokenizer, 'lang_code_to_id') else None

# Store language token IDs
LANG_TOKEN_IDS = {
    'ara_Arab': get_lang_token_id(translation_tokenizer, 'ara_Arab'),
    'fra_Latn': get_lang_token_id(translation_tokenizer, 'fra_Latn')
}

print(f"✅ Translation model loaded: {TRANSLATION_MODEL}")
print(f"  Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
print(f"  Language tokens: ara_Arab={LANG_TOKEN_IDS['ara_Arab']}, fra_Latn={LANG_TOKEN_IDS['fra_Latn']}")

Loading translation model...


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

✅ Translation model loaded: facebook/nllb-200-distilled-600M
  Device: cuda
  Language tokens: ara_Arab=3, fra_Latn=256057


In [6]:
def detect_language(text: str) -> str:
    """
    Detect if text is in Arabic or French
    
    Args:
        text: Input text
        
    Returns:
        'ar' for Arabic, 'fr' for French, 'unknown' for others
    """
    try:
        # Remove extra whitespace
        text = text.strip()
        
        # Check for Arabic characters
        arabic_chars = sum(1 for c in text if '\u0600' <= c <= '\u06FF')
        total_chars = len([c for c in text if c.isalpha()])
        
        if total_chars == 0:
            return 'unknown'
        
        # If more than 30% Arabic characters, it's Arabic
        if arabic_chars / total_chars > 0.3:
            return 'ar'
        
        # Use langdetect for French
        detected = detect(text)
        if detected == 'fr':
            return 'fr'
        elif detected == 'ar':
            return 'ar'
        else:
            # Default to French if uncertain (since data is in French)
            return 'fr' if arabic_chars == 0 else 'ar'
            
    except LangDetectException:
        return 'unknown'

def detect_output_language_intent(text: str) -> Optional[str]:
    """
    Detect if user explicitly requests output in specific language
    
    Args:
        text: User's question
        
    Returns:
        'ar' if Arabic output requested, 'fr' if French requested, None otherwise
    """
    text_lower = text.lower()
    
    # Arabic output request patterns in French
    french_to_arabic_patterns = [
        'en arabe',
        'réponds en arabe',
        'répondez en arabe',
        'traduis en arabe',
        'traduisez en arabe',
        'answer in arabic',
        'respond in arabic',
        'in arabic',
        'avec réponse en arabe',
        'réponse en arabe',
        'explique en arabe',
        'expliquez en arabe'
    ]
    
    # French output request patterns in Arabic
    arabic_to_french_patterns = [
        'بالفرنسية',
        'بالفرنساوية',
        'اجب بالفرنسية',
        'أجب بالفرنسية',
        'الجواب بالفرنسية',
        'en français',
        'in french',
        'réponds en français'
    ]
    
    # Check for Arabic output request
    for pattern in french_to_arabic_patterns:
        if pattern in text_lower:
            return 'ar'
    
    # Check for French output request
    for pattern in arabic_to_french_patterns:
        if pattern in text_lower:
            return 'fr'
    
    return None

def clean_question_for_retrieval(text: str, detected_intent: Optional[str]) -> str:
    """
    Remove language instruction phrases from question before retrieval
    
    Args:
        text: Original question
        detected_intent: Detected output language intent
        
    Returns:
        Cleaned question without language instructions
    """
    if detected_intent is None:
        return text
    
    # Patterns to remove
    removal_patterns = [
        # French patterns
        r'\s*en arabe\s*',
        r'\s*réponds en arabe\s*',
        r'\s*répondez en arabe\s*',
        r'\s*traduis en arabe\s*',
        r'\s*traduisez en arabe\s*',
        r'\s*answer in arabic\s*',
        r'\s*respond in arabic\s*',
        r'\s*in arabic\s*',
        r'\s*avec réponse en arabe\s*',
        r'\s*réponse en arabe\s*',
        r'\s*explique en arabe\s*',
        r'\s*expliquez en arabe\s*',
        # Arabic patterns
        r'\s*بالفرنسية\s*',
        r'\s*بالفرنساوية\s*',
        r'\s*اجب بالفرنسية\s*',
        r'\s*أجب بالفرنسية\s*',
        r'\s*الجواب بالفرنسية\s*',
        r'\s*en français\s*',
        r'\s*in french\s*',
        r'\s*réponds en français\s*'
    ]
    
    import re
    cleaned = text
    for pattern in removal_patterns:
        cleaned = re.sub(pattern, ' ', cleaned, flags=re.IGNORECASE)
    
    # Clean up extra spaces
    cleaned = ' '.join(cleaned.split())
    
    return cleaned

def translate_text(text: str, source_lang: str, target_lang: str) -> str:
    """
    Translate text between Arabic and French using NLLB-200
    
    Args:
        text: Text to translate
        source_lang: Source language code ('ar' or 'fr')
        target_lang: Target language code ('ar' or 'fr')
        
    Returns:
        Translated text
    """
    # NLLB-200 language codes
    lang_codes = {
        'ar': 'ara_Arab',
        'fr': 'fra_Latn'
    }
    
    src_code = lang_codes.get(source_lang, source_lang)
    tgt_code = lang_codes.get(target_lang, target_lang)
    
    # Set source language
    translation_tokenizer.src_lang = src_code
    
    # Tokenize input
    inputs = translation_tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )
    
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
    
    # Get target language token ID
    tgt_token_id = LANG_TOKEN_IDS.get(tgt_code)
    
    # Generate translation
    with torch.no_grad():
        if tgt_token_id is not None:
            translated_tokens = translation_model.generate(
                **inputs,
                forced_bos_token_id=tgt_token_id,
                max_length=512,
                num_beams=5,
                length_penalty=1.0,
                early_stopping=True
            )
        else:
            # Fallback without forced_bos_token_id
            translated_tokens = translation_model.generate(
                **inputs,
                max_length=512,
                num_beams=5,
                length_penalty=1.0,
                early_stopping=True
            )
    
    # Decode output
    translated_text = translation_tokenizer.batch_decode(
        translated_tokens,
        skip_special_tokens=True
    )[0]
    
    return translated_text

def translate_ar_to_fr(text: str) -> str:
    """Translate Arabic to French"""
    return translate_text(text, 'ar', 'fr')

def translate_fr_to_ar(text: str) -> str:
    """Translate French to Arabic"""
    return translate_text(text, 'fr', 'ar')

print("✅ Translation functions defined!")

# Test translation
test_ar = "ما هي الجزائر؟"
test_fr = translate_ar_to_fr(test_ar)
print(f"\nTranslation test:")
print(f"  Arabic: {test_ar}")
print(f"  French: {test_fr}")

✅ Translation functions defined!

Translation test:
  Arabic: ما هي الجزائر؟
  French: Quelle est l'Algérie ?


In [7]:
print("\nLoading embedding model...")
embedding_model = SentenceTransformer(EMBEDDING_MODEL)
embedding_dimension = embedding_model.get_sentence_embedding_dimension()

print(f"✅ Embedding model loaded!")
print(f"  Model: {EMBEDDING_MODEL}")
print(f"  Dimension: {embedding_dimension}")


Loading embedding model...


config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/400 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/809k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

✅ Embedding model loaded!
  Model: dangvantuan/sentence-camembert-large
  Dimension: 1024


In [8]:
print("Loading LLM for generation...")

tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)

if USE_4BIT_QUANTIZATION and torch.cuda.is_available():
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    llm_model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True
    )
    print(f"✅ LLM loaded with 4-bit quantization!")
else:
    llm_model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
        trust_remote_code=True
    )
    print(f"✅ LLM loaded!")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"  Device: {device}")
print(f"  Model: {LLM_MODEL}")

Loading LLM for generation...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

...

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

✅ LLM loaded with 4-bit quantization!
  Device: cuda
  Model: bofenghuang/vigogne-2-7b-chat


In [9]:
print("\nLoading existing index and documents...")

# Load FAISS index
index = faiss.read_index(INDEX_PATH)
print(f"✅ FAISS index loaded: {index.ntotal} vectors")

# Load documents
with open(DOCS_PATH, 'r', encoding='utf-8') as f:
    documents = json.load(f)
print(f"✅ Documents loaded: {len(documents)} documents")

# Load metadata
with open(META_PATH, 'r', encoding='utf-8') as f:
    metadata = json.load(f)
print(f"✅ Metadata loaded")
print(f"  Document types: {', '.join(metadata['document_types'])}")


Loading existing index and documents...
✅ FAISS index loaded: 18722 vectors
✅ Documents loaded: 18722 documents
✅ Metadata loaded
  Document types: jo_2020, code_civil, jo_2023, jo_2021, jo_2022, jo_2024, jo_2025, code_penale, constitution, code_commerce, code_famille


In [10]:
def retrieve_documents(query: str, top_k: int = 3) -> List[Tuple[Dict, float]]:
    """
    Retrieve most relevant documents for a query
    Query is always in French (translated if needed)
    """
    query_embedding = embedding_model.encode(
        [query],
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    
    scores, indices = index.search(query_embedding.astype('float32'), top_k)
    
    results = []
    for idx, score in zip(indices[0], scores[0]):
        if idx < len(documents):
            results.append((documents[idx], float(score)))
    
    return results

def generate_answer(query: str, context_docs: List[Tuple[Dict, float]]) -> str:
    """
    Generate answer using retrieved context
    Query and context are in French
    """
    # Build context
    context_parts = []
    for doc, score in context_docs:
        doc_type = doc.get('source_document_type', '').upper()
        header = doc.get('header', '')
        content = doc.get('content', '')
        
        ctx = f"[{doc_type}] {header}\n{content}"
        context_parts.append(ctx)
    
    context = "\n\n".join(context_parts)
    
    # Create prompt
    prompt = f"""<|system|>: Tu es un assistant juridique expert en droit algérien. Réponds de manière précise, professionnelle et factuelle en te basant strictement sur le contexte fourni. Si l'information n'est pas dans le contexte, indique-le clairement.
<|user|>: Contexte juridique:
{context}

Question: {query}
<|assistant|>:"""
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
    
    # Generate
    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )
    
    # Decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract answer
    if "<|assistant|>:" in response:
        answer = response.split("<|assistant|>:")[-1].strip()
    else:
        answer = response[len(prompt):].strip()
    
    return answer

def query_bilingual_rag(
    question: str,
    top_k: int = 3,
    show_sources: bool = True,
    force_output_lang: Optional[str] = None
) -> Dict:
    """
    Complete bilingual RAG query with smart language intent detection
    
    Args:
        question: User's question in Arabic or French (can include language instructions)
        top_k: Number of documents to retrieve
        show_sources: Whether to display sources
        force_output_lang: Force output language ('ar' or 'fr'), None for auto-detect
        
    Returns:
        Dictionary with answer, language info, and sources
    """
    print(f"\n{'='*70}")
    print(f"Question: {question}")
    
    # Detect if user explicitly requested specific output language
    output_lang_intent = detect_output_language_intent(question)
    
    # Detect input language
    input_lang = detect_language(question)
    print(f"Detected input language: {'Arabic' if input_lang == 'ar' else 'French' if input_lang == 'fr' else 'Unknown'}")
    
    if output_lang_intent:
        print(f"⚠️  User requested answer in: {'Arabic' if output_lang_intent == 'ar' else 'French'}")
    
    # Determine output language (priority: force > intent > input language)
    if force_output_lang:
        output_lang = force_output_lang
    elif output_lang_intent:
        output_lang = output_lang_intent
    else:
        output_lang = input_lang
    
    print(f"Output language: {'Arabic' if output_lang == 'ar' else 'French'}")
    print('='*70)
    
    # Clean question by removing language instruction phrases
    cleaned_question = clean_question_for_retrieval(question, output_lang_intent)
    if cleaned_question != question:
        print(f"\n🧹 Cleaned question: {cleaned_question}")
    
    # Translate query to French if needed (use cleaned version)
    french_query = cleaned_question
    if input_lang == 'ar':
        print("\n🔄 Translating query to French...")
        french_query = translate_ar_to_fr(cleaned_question)
        print(f"  Translated query: {french_query}")
    
    # Retrieve documents
    print("\n🔍 Retrieving relevant documents...")
    retrieved_docs = retrieve_documents(french_query, top_k)
    
    # Generate answer in French
    print("🤖 Generating answer...")
    french_answer = generate_answer(french_query, retrieved_docs)
    
    # Translate answer to Arabic if needed
    final_answer = french_answer
    if output_lang == 'ar':
        print("\n🔄 Translating answer to Arabic...")
        final_answer = translate_fr_to_ar(french_answer)
    
    # Display results
    print(f"\n📝 Answer:\n{final_answer}\n")
    
    if show_sources:
        print("📚 Sources:")
        for i, (doc, score) in enumerate(retrieved_docs, 1):
            print(f"\n  [{i}] {doc.get('source_document_type', '').upper()}")
            print(f"      {doc.get('header', '')}")
            print(f"      Relevance: {score:.4f}")
            content = doc.get('content', '')
            preview = content[:200] + "..." if len(content) > 200 else content
            print(f"      {preview}")
    
    return {
        "question": question,
        "cleaned_question": cleaned_question,
        "input_language": input_lang,
        "output_language": output_lang,
        "language_intent_detected": output_lang_intent is not None,
        "french_query": french_query if input_lang == 'ar' else None,
        "french_answer": french_answer if output_lang == 'ar' else None,
        "answer": final_answer,
        "sources": [(doc, score) for doc, score in retrieved_docs]
    }

print("✅ Bilingual RAG functions defined!")

✅ Bilingual RAG functions defined!


In [11]:
print("\n" + "="*70)
print("TESTING FRENCH QUERIES")
print("="*70)

result_fr = query_bilingual_rag(
    "Qu'est-ce que l'Algérie selon la constitution?",
    top_k=3
)


TESTING FRENCH QUERIES

Question: Qu'est-ce que l'Algérie selon la constitution?
Detected input language: French
Output language: French

🔍 Retrieving relevant documents...
🤖 Generating answer...

📝 Answer:
Selon la Constitution de l'Algérie, l'Algérie est une République Démocratique et Populaire. Elle est considérée comme une entité unique et indivisible. La capitale de la République est Alger, telle qu'établie par l'article 1 du Chapitre Premier de la Constitution. En ce qui concerne la nationalité, l'article 41 du Chapitre II du Livret de Famille et les Fiches d'État Civil de l'Euxieme déclaration de la représentation légale mentionne que "l'expression 'en Algérie' s'entend de tout le territoire algérien, des eaux territoriales algériennes, des navires et aéronefs algériens."

📚 Sources:

  [1] CONSTITUTION
      Title I - Chapter PREMIER - Article 1er
      Relevance: 0.4948
      L'Algérie est une République Démocratique et Populaire. Elle est une et indivisible.

  [2] CONSTITUT

In [12]:
print("\n" + "="*70)
print("TESTING ARABIC QUERIES")
print("="*70)

result_ar = query_bilingual_rag(
    "ما هي الجزائر حسب الدستور؟",
    top_k=3
)


TESTING ARABIC QUERIES

Question: ما هي الجزائر حسب الدستور؟
Detected input language: Arabic
Output language: Arabic

🔄 Translating query to French...
  Translated query: Qu'est-ce que l'Algérie selon la Constitution ?

🔍 Retrieving relevant documents...
🤖 Generating answer...

🔄 Translating answer to Arabic...

📝 Answer:
️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️

📚 Sources:

  [1] CONSTITUTION
      Title I - Chapter PREMIER - Article 1er
      Relevance: 0.4992
      L'Algérie est une République Démocratique et Populaire. Elle est une et indivisible.

  [2] CONSTITUTION
      Title I - Chapter PREMIER - Article 5
      Relevance: 0.4797
      La capitale de la République est Alger.

  [3] CODE_CIVIL
      EUXIEME DE LA REPRESENTATION LEGALE - LE LIVRET DE FAMILLE ET LES FICHES D'ETAT CIVIL - Article 5
      Relevance: 0.4683
      Arti

In [13]:
print("\n" + "="*70)
print("TESTING LANGUAGE INTENT DETECTION")
print("="*70)

# Test 1: French query with "en arabe" instruction
print("\n--- Test 1: French query requesting Arabic answer ---")
result1 = query_bilingual_rag(
    "Qu'est-ce que l'Algérie selon la constitution? Réponds en arabe",
    top_k=3
)

print("\n" + "="*70)

# Test 2: French query with "answer in arabic"
print("\n--- Test 2: French query with English instruction ---")
result2 = query_bilingual_rag(
    "Quels sont les droits fondamentaux? Answer in Arabic",
    top_k=3
)

print("\n" + "="*70)

# Test 3: Arabic query with French output request
print("\n--- Test 3: Arabic query requesting French answer ---")
result3 = query_bilingual_rag(
    "ما هي الحريات الأساسية؟ الجواب بالفرنسية",
    top_k=3
)

print("\n" + "="*70)

# Test 4: More natural phrasing
print("\n--- Test 4: Natural language instruction ---")
result4 = query_bilingual_rag(
    "Explique-moi le système législatif algérien, avec réponse en arabe s'il te plaît",
    top_k=3
)

print("\n" + "="*70)

# Test 5: No language instruction (should use input language)
print("\n--- Test 5: No language instruction ---")
result5 = query_bilingual_rag(
    "Quelle est la capitale de l'Algérie?",
    top_k=3
)



TESTING LANGUAGE INTENT DETECTION

--- Test 1: French query requesting Arabic answer ---

Question: Qu'est-ce que l'Algérie selon la constitution? Réponds en arabe
Detected input language: French
⚠️  User requested answer in: Arabic
Output language: Arabic

🧹 Cleaned question: Qu'est-ce que l'Algérie selon la constitution? Réponds

🔍 Retrieving relevant documents...
🤖 Generating answer...

🔄 Translating answer to Arabic...

📝 Answer:
️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️

📚 Sources:

  [1] CONSTITUTION
      Title I - Chapter PREMIER - Article 1er
      Relevance: 0.4676
      L'Algérie est une République Démocratique et Populaire. Elle est une et indivisible.

  [2] CONSTITUTION
      Title I - Chapter PREMIER - Article 5
      Relevance: 0.4389
      La capitale de la République est A